# Build the Neural Network
[教程链接](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

In [114]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## device

In [115]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cuda


## Define the Class

In [116]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [117]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits.size())
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

torch.Size([1, 10])
tensor([6], device='cuda:0')


## Model Layers

In [118]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten()

In [119]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [120]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [121]:
print(hidden1)
hidden1 = nn.ReLU()(hidden1)
print(hidden1)

tensor([[ 0.0893,  0.6335, -0.1976, -0.5442, -0.1960, -0.4315,  0.6139, -0.2039,
         -0.0648, -0.1255,  0.0763, -0.9271, -0.4055,  0.5128, -0.6723,  0.3361,
          0.3317,  0.7255, -0.0107,  0.1347],
        [ 0.2374,  0.3167, -0.0260, -0.1440, -0.0900, -0.3333,  0.0976, -0.1344,
         -0.1646, -0.2335, -0.1101, -0.5949, -0.4035,  0.3730, -0.4570,  0.5315,
          0.4057, -0.0257, -0.0462, -0.0703],
        [-0.1246,  0.3470, -0.4149, -0.1737, -0.1363, -0.1687,  0.3957, -0.4341,
          0.1361, -0.3473, -0.2777, -0.7721, -0.3793, -0.0617, -0.5624,  0.6923,
          0.5330,  0.1833, -0.3382, -0.0070]], grad_fn=<AddmmBackward0>)
tensor([[0.0893, 0.6335, 0.0000, 0.0000, 0.0000, 0.0000, 0.6139, 0.0000, 0.0000,
         0.0000, 0.0763, 0.0000, 0.0000, 0.5128, 0.0000, 0.3361, 0.3317, 0.7255,
         0.0000, 0.1347],
        [0.2374, 0.3167, 0.0000, 0.0000, 0.0000, 0.0000, 0.0976, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.3730, 0.0000, 0.5315, 0.4057, 0.0000,

### nn.Sequential

In [122]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits.size())
logits

torch.Size([3, 10])


tensor([[-0.2352, -0.2778, -0.1246, -0.1669,  0.0826, -0.0531, -0.0434, -0.1218,
         -0.0860, -0.2808],
        [-0.0992, -0.1905, -0.1495,  0.0723,  0.0470, -0.0925, -0.2366, -0.0185,
         -0.0081, -0.2828],
        [-0.1344, -0.2732, -0.1670, -0.0695,  0.0744, -0.1595, -0.1871, -0.0918,
         -0.0067, -0.2947]], grad_fn=<AddmmBackward0>)

### nn.Softmax

In [123]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0895, 0.0858, 0.1000, 0.0959, 0.1230, 0.1074, 0.1085, 0.1003, 0.1040,
         0.0856],
        [0.0990, 0.0904, 0.0942, 0.1176, 0.1146, 0.0997, 0.0863, 0.1073, 0.1085,
         0.0824],
        [0.0991, 0.0862, 0.0959, 0.1057, 0.1221, 0.0966, 0.0940, 0.1034, 0.1126,
         0.0844]], grad_fn=<SoftmaxBackward0>)

## Model Parameters

In [124]:
print(model)
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0285,  0.0162,  0.0355,  ..., -0.0274,  0.0306, -0.0347],
        [-0.0039,  0.0353, -0.0045,  ...,  0.0240, -0.0260,  0.0228]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0043,  0.0074], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0068, -0.0260, -0.0098,  ...,  0.0299, -0.0395,  0.0009],
        [-0.0298, -0.0286, -0.0413,  ...,  0.0405,  0.0220, -0.0080]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

La